In [1]:
# for stable version 
# pip install invana

# for installation specific branch
#pip install git+https://github.com/invana/invana@refactor/invana-connectors#egg=invana

# for latest code 
import sys
sys.path.append("../")

In [2]:
from invana import InvanaGraph

gremlin_server_url = "ws://megamind.local:8182/gremlin"
graph = InvanaGraph(gremlin_server_url)

[DEBUG:2023-01-25 00:26:52,841:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTING
[DEBUG:2023-01-25 00:26:52,842:connector._init_connection:99] - create driver connection  
[DEBUG:2023-01-25 00:26:53,849:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTED


In [3]:
from invana.ogm.models import VertexModel, EdgeModel
from invana.ogm.fields import StringProperty, DateTimeProperty, IntegerProperty, FloatProperty, BooleanProperty
from invana.ogm import indexes
from datetime import datetime


class Person(VertexModel):
    graph = graph
    properties = {
        'first_name': StringProperty(max_length=30, trim_whitespaces=True),
        'is_active': BooleanProperty(default=True),
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }

class Roles(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty()
    }

class Programming(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty(min_length=2, trim_whitespaces=True)
    }
    
class Game(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty(min_length=2, trim_whitespaces=True)
    }
        
class HasSkill(EdgeModel):
    graph = graph
    properties = {
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }

class HasRole(EdgeModel):
    graph = graph
    properties = {
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }
    
    

In [4]:
from invana.helpers.helpers import install_models

shoud_install_models = True

if shoud_install_models is True:
    # i_understand_rollback=True will roll back any transactions in pending.
    install_models(graph, Person, Programming, Game, HasSkill, Roles, HasRole)
    print("Installed models")
else:
    print(f"Ignoring models installation because install_models={install_models}")

[DEBUG:2023-01-25 00:26:53,889:events.log_event:55] - Request ad35f395-babf-4d32-9fcc-9a2beb4eece2 STARTED with query: mgmt = graph.openManagement()
if (mgmt.containsVertexLabel('Person'))
    Person = mgmt.getVertexLabel('Person')
else 
    Person = mgmt.makeVertexLabel('Person').make()
if (mgmt.containsRelationType('first_name'))
    first_name = mgmt.getPropertyKey('first_name')
else 
    first_name = mgmt.makePropertyKey('first_name').dataType(String.class).make()
if (mgmt.containsRelationType('is_active'))
    is_active = mgmt.getPropertyKey('is_active')
else 
    is_active = mgmt.makePropertyKey('is_active').dataType(Boolean.class).make()
if (mgmt.containsRelationType('created_at'))
    created_at = mgmt.getPropertyKey('created_at')
else 
    created_at = mgmt.makePropertyKey('created_at').dataType(Date.class).make()
mgmt.addProperties(Person, first_name, is_active, created_at)
mgmt.commit();; request_options: {};; at 2023-01-25 00:26:53.889132
[DEBUG:2023-01-25 00:26:58,929:even

Installed models


In [5]:
Person.objects.delete()
Programming.objects.delete()
Game.objects.delete()
HasSkill.objects.delete()
HasRole.objects.delete()

p = Person.objects.create(first_name="rrmerugu")

_, django = Programming.objects.get_or_create(name="Django")
_, python = Programming.objects.get_or_create(name="Python")
_, neo4j = Programming.objects.get_or_create(name="neo4j")
_, janusgraph = Programming.objects.get_or_create(name="Janusgraph")

HasSkill.objects.get_or_create(p.id, django.id)
HasSkill.objects.get_or_create(p.id, python.id)
HasSkill.objects.get_or_create(p.id, neo4j.id)
HasSkill.objects.get_or_create(p.id, janusgraph.id)

_, cod = Game.objects.get_or_create(name="Call of Duty")
_, fifa = Game.objects.get_or_create(name="Fifa")

HasSkill.objects.get_or_create(p.id, cod.id)
HasSkill.objects.get_or_create(p.id, fifa.id)


_, dev_role = Roles.objects.get_or_create(name="Developer")
_, admin_role = Roles.objects.get_or_create(name="Admin")

HasRole.objects.get_or_create(p.id, dev_role.id)
HasRole.objects.get_or_create(p.id, admin_role.id)


print("Created data")

Created data


In [6]:
# p = Person.objects.get_or_none(has__first_name="rrmerugu")
# graph.connector.vertex.search(has__id=p.id).get_traversal() \
# .bothE("has_skill")._as("e") \
# .inV().hasLabel("Programming")._as("v")\
# .select("a","e") \
# .toList()
# # print(p)

In [7]:
from gremlin_python.process.graph_traversal import __

p = Person.objects.get_or_none(has__first_name="rrmerugu")
data = graph.connector.vertex.search(has__id=p.id).get_traversal() \
.bothE("has_skill") \
.inV().hasLabel("Programming") \
.path().by(__.elementMap())
#.toList()

for d in data:
    print("===", d)
# print(p)

=== path[<Node:Person id="41095280" first_name=rrmerugu created_at=2023-01-25 00:27:14.204000 is_active=True >, <RelationShip:has_skill id="ofjf2-ogtbk-28lx-18d4" 41095280:Person -> has_skill -> 57496:Programming created_at=2023-01-25 00:27:14.428000 >, <Node:Programming id="57496" name=neo4j >]
=== path[<Node:Person id="41095280" first_name=rrmerugu created_at=2023-01-25 00:27:14.204000 is_active=True >, <RelationShip:has_skill id="ofjta-ogtbk-28lx-1en4" 41095280:Person -> has_skill -> 65632:Programming created_at=2023-01-25 00:27:14.443000 >, <Node:Programming id="65632" name=Janusgraph >]
=== path[<Node:Person id="41095280" first_name=rrmerugu created_at=2023-01-25 00:27:14.204000 is_active=True >, <RelationShip:has_skill id="ofj0u-ogtbk-28lx-1rbk" 41095280:Person -> has_skill -> 82064:Programming created_at=2023-01-25 00:27:14.412000 >, <Node:Programming id="82064" name=Python >]
=== path[<Node:Person id="41095280" first_name=rrmerugu created_at=2023-01-25 00:27:14.204000 is_active

In [8]:
p = Person.objects.get_or_none(has__first_name="rrmerugu")
graph.connector.vertex.getNodeInComingNeighbors(p.id).toList()

[]

In [9]:
p = Person.objects.get_or_none(has__first_name="rrmerugu")
graph.connector.vertex.getNodeOutGoingNeighbors(p.id, "has_role").toList()

[path[<Node:Person id="41095280" first_name=rrmerugu created_at=2023-01-25 00:27:14.204000 is_active=True >, <RelationShip:has_role id="ofm6m-ogtbk-2brp-m74" 41095280:Person -> has_role -> 28768:Roles created_at=2023-01-25 00:27:14.643000 >, <Node:Roles id="28768" name=Developer >],
 path[<Node:Person id="41095280" first_name=rrmerugu created_at=2023-01-25 00:27:14.204000 is_active=True >, <RelationShip:has_role id="ofmku-ogtbk-2brp-sl4" 41095280:Person -> has_role -> 37048:Roles created_at=2023-01-25 00:27:14.659000 >, <Node:Roles id="37048" name=Admin >]]

In [10]:
p = Person.objects.get_or_none(has__first_name="rrmerugu")
graph.connector.vertex.getNodeOutGoingNeighbors(p.id, "has_skill", neighbors_labels=["Game"]).toList()

[path[<Node:Person id="41095280" first_name=rrmerugu created_at=2023-01-25 00:27:14.204000 is_active=True >, <RelationShip:has_skill id="ofle6-ogtbk-28lx-157s" 41095280:Person -> has_skill -> 53416:Game created_at=2023-01-25 00:27:14.596000 >, <Node:Game id="53416" name=Call of Duty >],
 path[<Node:Person id="41095280" first_name=rrmerugu created_at=2023-01-25 00:27:14.204000 is_active=True >, <RelationShip:has_skill id="oflse-ogtbk-28lx-ogwhc" 41095280:Person -> has_skill -> 41099376:Game created_at=2023-01-25 00:27:14.613000 >, <Node:Game id="41099376" name=Fifa >]]

In [11]:
# p = Person.objects.get_or_none(has__first_name="rrmerugu")
# graph.connector.vertex.search(has__id=p.id).get_traversal().outE().inV().path().by(__.elementMap()).toList()

In [12]:
p = Programming.objects.get_or_none(has__name="Django")

graph.connector.vertex.getNodeInComingNeighbors(p.id,).toList()        


[path[<Node:Programming id="118800" name=Django >, <RelationShip:has_skill id="ofimm-ogtbk-28lx-2jo0" 41095280:Person -> has_skill -> 118800:Programming created_at=2023-01-25 00:27:14.396000 >, <Node:Person id="41095280" first_name=rrmerugu created_at=2023-01-25 00:27:14.204000 is_active=True >]]